In [1]:
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import time
import datetime
import os

# client_id='e08812d0ad874b53bb01e49ada6166ad'
# client_secret='9085378d43d24d6c823a92c0f2cb03b5'

# # # 민진님
# # client_id = "7a98ceda33624c9bb15bf7f877176e50"
# # client_secret = "74fb60bce9d048e3a9faa52427e6d0f5"

# # 세인님
# client_id = "05c96c56746e49b9b26e2ccf9837c7b7"
# client_secret = "98238151cd5e46df93049946ddd3673a"

# 서브 1
client_id = "55cc1bbf202640ee8a691ecab9be6e1d"
client_secret = "ef892d15d19543f4840c5ff069e6503c"

# # 서브 2
# client_id = "37211c9678ae40e6aed26be454135b1e"
# client_secret = "031a960bb8bc4367a412f63d25f1f513"

# # 서브 3
# client_id = "4cda4cd5cf9a4877b55336bedbe21c9c"
# client_secret = "93bacd33bd494ec5bc76034f3705d502"

# # 서브 4
# client_id = "9e6e9e6622204dbe85e2553c9b026ed5"
# client_secret = "b7b96ff10fae4077ab8912e1fb9d1f6b"

# # 서브 5
# client_id = "8e564786163d4a66a8e8d844955b9c2d"
# client_secret = "7966e74a259d4fd9a3d689b54fd3d100"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
start_time = time.time()

year_list = ['2023']
for year in year_list:  
      
    genre_list = ['k-pop']
    for genre in genre_list:
        
        artist_name =[]
        track_name = []
        track_popularity =[]
        artist_id =[]
        track_id =[]     
        release_date = []
        album_image = []

        
        for i in range(0,50,50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
                release_date.append(t['album']['release_date'])

                try:
                    album_image.append(t['album']['images'][1]['url'])  # 320 사이즈 이미지
                except IndexError:
                    album_image.append(None)
                
                
                
# 트랙 데이터
# artist_id,            artist_name,            track_id,          track_name,           release_date,         track_popularity,         album_image
#track_df = pd.DataFrame({'artist_id' : artist_id, 'artist_name' : artist_name, 'track_id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity })
track_df = pd.DataFrame({'artist_id' : artist_id, 'artist_name' : artist_name, 'track_id' : track_id, 'track_name' : track_name,  'release_date': release_date, 'track_popularity': track_popularity,'album_image': album_image }) 

# 트랙 csv 파일 저장 
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
track_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_track_{timestamp}.csv', index=False)


# 아티스트 파일을 위한 1차 컬럼
track_df2 = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name})

print("오디오 피쳐 2분 대기")
time.sleep(10)

artist_popularity = []
artist_genres = []
artist_followers =[]

for a_id in track_df.artist_id:
    artist = sp.artist(a_id)
    artist_popularity.append(artist['popularity'])
    artist_genres.append(artist['genres'])
    artist_followers.append(artist['followers']['total'])

# 아티스트 파일을 위한 최정 컬럼 ( 이걸로 csv 파일 만들기)
track_df3 = track_df2.assign(artist_popularity=artist_popularity, artist_followers=artist_followers, artist_genres=artist_genres)

# 아티스트 csv 파일 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
track_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_artist_{timestamp}.csv', index=False)


# 오디오 특징을 위한 1차 컬럼
track_df = track_df.assign(artist_popularity=artist_popularity, artist_genres=artist_genres, artist_followers=artist_followers)


print("오디오 피쳐 5분 대기")
time.sleep(10)

track_features = []
for t_id in track_df['track_id']:
    af = sp.audio_features(t_id)
    track_features.append(af)
    
tf_df = pd.concat([pd.DataFrame.from_records(item) for item in track_features], ignore_index=True)

tf_df = tf_df.rename(columns={'id': 'track_id'})

# 오디오 특징을 위한 컬럼 ( 이걸로 csv 파일 만들기 )
tf_df = tf_df[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'time_signature']]
# 오디오 csv 파일 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
tf_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_audio_features_{timestamp}.csv', index=False)

#tf_df.head()       

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")
     


classical, study, sad, kids, jazz


genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']
genre_list 에 있는것을  for문 통해 16번을 실행한 결과를  하나의 track_df로 묶은 다음에
컬럼 track_id를 기준으로 중복된 값을 하나만 남기고 제거 할 수 있을까??????????  흠............

In [ ]:
start_time = time.time()

year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010','2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop','party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']

for year in year_list:
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
                
        # 트랙 파일을 위한 컬럼 (이걸로 CSV 파일 만들기)
        track_df = pd.DataFrame({'artist_name': artist_name, 'track_name': track_name, 'track_id': track_id, 'track_popularity': track_popularity, 'artist_id': artist_id })

        # 트랙 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        track_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_track_{timestamp}.csv', index=False)


In [ ]:
#### 성공 코드 트랙 기준으로 가수 중복 제거한 상태로 다운로드 하는 코드 ( 장르별 중복은 아직 안됨 )

start_time = time.time()

year_list = ['2023', '2022'] # 테스트용
#year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
for year in year_list:
    
    genre_list = ['k-pop', 'acoustic'] # 테스트용
    #genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop', 'party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
                
        # 트랙 파일을 위한 컬럼 (이걸로 CSV 파일 만들기)
        track_df = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'track_name': track_name, 'track_id': track_id, 'track_popularity': track_popularity})
        
        # 트랙 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        track_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_track_{timestamp}.csv', index=False)
        
        # 아티스트 파일을 위한 1차 컬럼 필터링
        track_df2 = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name})
        
        print("오디오 피쳐 2분 대기")
        time.sleep(10)
        
        artist_popularity = []
        artist_genres = []
        artist_followers = []
        
        for a_id in track_df.artist_id:
            artist = sp.artist(a_id)
            artist_popularity.append(artist['popularity'])
            artist_genres.append(artist['genres'])
            artist_followers.append(artist['followers']['total'])
        
        # 아티스트 파일을 위한 최종 컬럼 (이걸로 CSV 파일 만들기)
        track_df3 = track_df2.assign(artist_popularity=artist_popularity, artist_followers=artist_followers, artist_genres=artist_genres)
        
        # 아티스트 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        track_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_artist_{timestamp}.csv', index=False)
        
        print("오디오 피쳐 5분 대기")
        time.sleep(10)
        
        track_features = []
        for t_id in track_df['track_id']:
            af = sp.audio_features(t_id)
            track_features.append(af)
            
        tf_df = pd.concat([pd.DataFrame.from_records(item) for item in track_features], ignore_index=True)
        
        tf_df = tf_df.rename(columns={'id': 'track_id'})
        
        # 오디오 특징을 위한 컬럼 (이걸로 CSV 파일 만들기)
        tf_df = tf_df[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'time_signature']]
        
        # 오디오 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        tf_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_audio_features_{timestamp}.csv', index=False)
        
end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


In [ ]:
start_time = time.time()

year_list = ['2023', '2022']
# year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
for year in year_list:
    
    genre_list = ['k-pop', 'acoustic']
    #genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop', 'party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']
    
    for genre in genre_list:
        
        print(f"{year}_{genre}: 다운로드")
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
                
        # 트랙 파일을 위한 컬럼 (이걸로 CSV 파일 만들기)
        track_df = pd.DataFrame({'artist_id': artist_id,'artist_name': artist_name, 'track_name': track_name, 'track_id': track_id, 'track_popularity': track_popularity})  
        
        # 트랙 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        track_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_track_{timestamp}.csv', index=False)
        
        # 아티스트 파일을 위한 1차 컬럼
        track_df2 = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name})
        
        print("오디오 피쳐 2분 대기")
        time.sleep(10)
        
        artist_popularity = []
        artist_genres = []
        artist_followers = []
        
        for a_id in track_df.artist_id:
            artist = sp.artist(a_id)
            artist_popularity.append(artist['popularity'])
            artist_genres.append(artist['genres'])
            artist_followers.append(artist['followers']['total'])
        
        # 아티스트 파일을 위한 최종 컬럼 (이걸로 CSV 파일 만들기)
        track_df3 = track_df2.assign(artist_popularity=artist_popularity, artist_followers=artist_followers, artist_genres=artist_genres)
        
        # 아티스트 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        track_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_artist_{timestamp}.csv', index=False)
        
        print("오디오 피쳐 5분 대기")
        time.sleep(10)
        
        track_features = []
        for t_id in track_df['track_id']:
            af = sp.audio_features(t_id)
            track_features.append(af)
            
        tf_df = pd.concat([pd.DataFrame.from_records(item) for item in track_features], ignore_index=True)
        
        tf_df = tf_df.rename(columns={'id': 'track_id'})
        
        # 오디오 특징을 위한 컬럼 (이걸로 CSV 파일 만들기)
        tf_df = tf_df[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'time_signature']]
        
        # 오디오 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        tf_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_audio_features_{timestamp}.csv', index=False)
        
end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


In [5]:
start_time = time.time()

year_list = ['2023', '2022']
# year_list = ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000']
for year in year_list:
    
    genre_list = ['k-pop', 'acoustic']
    #genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop', 'party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']
    
    for genre in genre_list:
        
        print(f"{year}_{genre}: 다운로드")
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
                
        # 트랙 파일을 위한 컬럼 (이걸로 CSV 파일 만들기)
        track_df = pd.DataFrame({'artist_name': artist_name, 'track_name': track_name, 'track_id': track_id, 'track_popularity': track_popularity, 'artist_id': artist_id})  
        
        # 트랙 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        track_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_track222_{timestamp}.csv', index=False)

2023_k-pop: 다운로드


KeyboardInterrupt: 

In [ ]:
#########################

In [ ]:
start_time = time.time()

temporary_df = pd.DataFrame()  # 빈 DataFrame 생성

year_list = ['2023']
for year in year_list:
    genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop', 'party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul'] # 16
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")       
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []      
        for i in range(0, 100, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
        
        track_df = pd.DataFrame({ 'artist_id': artist_id, 'artist_name': artist_name, 'track_name': track_name, 'track_id': track_id, 'track_popularity': track_popularity})
        
        temporary_df = pd.concat([temporary_df, track_df])  # temporary_df와 track_df를 병합

# 중복이 제거된 DataFrame 저장
temporary_df.drop_duplicates(subset='track_id', inplace=True)  # track_id를 기준으로 중복 제거

# 지정한 데이터 프레임 형식으로 CSV 파일 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/combined_tracks_{timestamp}.csv', index=False)

end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


2023_k-pop: 다운로드


In [64]:
temporary_df.head(10)

,artist_id,artist_name,track_name,track_id,track_popularity
0,2LIk90788K0zvyj2JJVwkJ,Jack Harlow,Lovin On Me,4xhsWYTOGcal8zt0J161CU,96
1,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,Standing Next to You,2KslE17cAJNHTsI2MI0jb2,98
2,1oSPZhvZMIrWW5I41kPkkY,Jimin,Like Crazy,3Ua0m0YmEjrMi9XErKcNiR,94
3,4SpbR6yFEvexJuaBpgAU5p,LE SSERAFIM,Perfect Night,74X2u8JMVooG2QbjRxXwR8,90
4,1oSPZhvZMIrWW5I41kPkkY,Jimin,Like Crazy,58BWnoFDNaeM0vwA0K3fqb,90
5,6HvZYsbFfjnjFrWF950C9d,NewJeans,Super Shy,5sdQOyqq2IDhvmx2lHOpwd,90
6,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,Seven (feat. Latto) (Explicit Ver.),7x9aauaA9cu6tyfpHnqDLo,92
7,2dIgFjalVxs4ThymZ67YCE,Stray Kids,LALALALA,1OG1NoKpZZLrMqMYCk9m84,87
8,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,3D (feat. Jack Harlow),6ehWdR7cGDXnT7aKEASJxE,90
9,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,Yes or No,2gkVEnpahpE3bQuvGuCpAV,90


In [65]:
track_df2 = temporary_df[['artist_id', 'artist_name']]

In [66]:
track_df2.head(10)

,artist_id,artist_name
0,2LIk90788K0zvyj2JJVwkJ,Jack Harlow
1,6HaGTQPmzraVmaVxvz6EUc,Jung Kook
2,1oSPZhvZMIrWW5I41kPkkY,Jimin
3,4SpbR6yFEvexJuaBpgAU5p,LE SSERAFIM
4,1oSPZhvZMIrWW5I41kPkkY,Jimin
5,6HvZYsbFfjnjFrWF950C9d,NewJeans
6,6HaGTQPmzraVmaVxvz6EUc,Jung Kook
7,2dIgFjalVxs4ThymZ67YCE,Stray Kids
8,6HaGTQPmzraVmaVxvz6EUc,Jung Kook
9,6HaGTQPmzraVmaVxvz6EUc,Jung Kook


In [ ]:
artist_popularity = []
artist_genres = []
artist_followers =[]
for a_id in track_df.artist_id:
    artist = sp.artist(a_id)
    artist_popularity.append(artist['popularity'])
    artist_genres.append(artist['genres'])
    artist_followers.append(artist['followers']['total'])


In [ ]:
start_time = time.time()

temporary_df = pd.DataFrame()  # 빈 DataFrame 생성

year_list = ['2023', '2022', '2021', '2020']
for year in year_list:
    genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop', 'party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']
    for genre in genre_list:
        print(f"{year}_{genre}: 다운로드")       
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []      
        for i in range(0, 100, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
        
        track_df = pd.DataFrame({
            'artist_name': artist_name,
            'track_name': track_name,
            'track_id': track_id,
            'track_popularity': track_popularity,
            'artist_id': artist_id
        })
        
        temporary_df = pd.concat([temporary_df, track_df])  # temporary_df와 track_df를 병합

# 중복이 제거된 DataFrame 저장
temporary_df.drop_duplicates(subset='track_id', inplace=True)  # track_id를 기준으로 중복 제거

# 지정한 데이터 프레임 형식으로 CSV 파일 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/combined_tracks_{timestamp}.csv', index=False)

# 아티스트 파일을 위한 1차 컬럼 필터링
track_df2 = temporary_df[['artist_id', 'artist_name']].drop_duplicates(subset='artist_id')

artist_popularity = []
artist_genres = []
artist_followers = []

for a_id in temporary_df.artist_id:
    artist = sp.artist(a_id)
    artist_popularity.append(artist['popularity'])
    artist_genres.append(artist['genres'])
    artist_followers.append(artist['followers']['total'])

# 아티스트 파일을 위한 최종 컬럼 (이걸로 CSV 파일 만들기)
track_df3 = track_df2.assign(artist_popularity=artist_popularity, artist_followers=artist_followers, artist_genres=artist_genres)

# 아티스트 CSV 파일 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
track_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_test_{timestamp}.csv', index=False)


end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")

In [44]:
track_df2.head()

,artist_id,artist_name
0,2LIk90788K0zvyj2JJVwkJ,Jack Harlow
1,6HaGTQPmzraVmaVxvz6EUc,Jung Kook
2,1oSPZhvZMIrWW5I41kPkkY,Jimin
3,4SpbR6yFEvexJuaBpgAU5p,LE SSERAFIM
5,6HvZYsbFfjnjFrWF950C9d,NewJeans


In [ ]:
artist_popularity = []
artist_genres = []
artist_followers = []

for a_id in temporary_df.artist_id:
    artist = sp.artist(a_id)
    artist_popularity.append(artist['popularity'])
    artist_genres.append(artist['genres'])
    artist_followers.append(artist['followers']['total'])

# 아티스트 파일을 위한 최종 컬럼 (이걸로 CSV 파일 만들기)
track_df3 = track_df2.assign(artist_popularity=artist_popularity, artist_followers=artist_followers, artist_genres=artist_genres)

# 아티스트 CSV 파일 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
track_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_test_{timestamp}.csv', index=False)


In [ ]:
start_time = time.time()

temporary_df = pd.DataFrame()  # 빈 DataFrame 생성

year_list = ['2023', '2022', '2021', '2020']
for year in year_list:
    genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop', 'party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']
    for genre in genre_list:
        
        print(f"{year}_{genre}: 다운로드")
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []
        
        for i in range(0, 5, 5):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
                
        # 트랙 파일을 위한 컬럼 (이걸로 CSV 파일 만들기)
        track_df = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'track_name': track_name, 'track_id': track_id, 'track_popularity': track_popularity})
        
        # temporary_df와 track_df를 병합
        temporary_df = pd.concat([temporary_df, track_df])
        
        # 중복이 제거된 DataFrame 저장
        temporary_df.drop_duplicates(subset='track_id', inplace=True)  # track_id를 기준으로 중복 제거
        
        # 트랙 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_track_{timestamp}.csv', index=False)
        
        # 아티스트 파일을 위한 1차 컬럼 필터링
        track_df2 = temporary_df[['artist_id', 'artist_name']]
        
        print("오디오 피쳐 2분 대기")
        time.sleep(10)
        
        artist_popularity = []
        artist_genres = []
        artist_followers = []
        
        for a_id in track_df.artist_id:
            artist = sp.artist(a_id)
            artist_popularity.append(artist['popularity'])
            artist_genres.append(artist['genres'])
            artist_followers.append(artist['followers']['total'])
        
        # 아티스트 파일을 위한 최종 컬럼 (이걸로 CSV 파일 만들기)
        track_df3 = track_df2.assign(artist_popularity=artist_popularity, artist_followers=artist_followers, artist_genres=artist_genres)
        
        # 아티스트 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        track_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_artist_{timestamp}.csv', index=False)
        
        print("오디오 피쳐 5분 대기")
        time.sleep(10)
        
        track_features = []
        for t_id in track_df['track_id']:
            af = sp.audio_features(t_id)
            track_features.append(af)
            
        tf_df = pd.concat([pd.DataFrame.from_records(item) for item in track_features], ignore_index=True)
        
        tf_df = tf_df.rename(columns={'id': 'track_id'})
        
        # 오디오 특징을 위한 컬럼 (이걸로 CSV 파일 만들기)
        tf_df = tf_df[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'time_signature']]
        
        # 오디오 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        tf_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_audio_features_{timestamp}.csv', index=False)
        
end_time = time.time()
print(f"작업 소요 시간: {end_time - start_time} 초")


In [49]:
start_time = time.time()

temporary_df = pd.DataFrame()  # 빈 DataFrame 생성

year_list = ['2023', '2022', '2021', '2020']
for year in year_list:
    genre_list = ['k-pop', 'acoustic', 'club', 'dance', 'edm', 'disco', 'electronic', 'happy', 'hip-hop', 'party', 'pop', 'r-n-b', 'rock', 'romance', 'sleep', 'soul']
    for genre in genre_list:
        
        print(f"{year}_{genre}: 다운로드")
        
        artist_name = []
        track_name = []
        track_popularity = []
        artist_id = []
        track_id = []
        
        for i in range(0, 50, 50):
            track_results = sp.search(q=f'year:{year} genre:{genre}', type='track', limit=50, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                artist_name.append(t['artists'][0]['name'])
                artist_id.append(t['artists'][0]['id'])
                track_name.append(t['name'])
                track_id.append(t['id'])
                track_popularity.append(t['popularity'])
                
        # 트랙 파일을 위한 컬럼 (이걸로 CSV 파일 만들기)
        track_df = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'track_name': track_name, 'track_id': track_id, 'track_popularity': track_popularity})
        
        # temporary_df와 track_df를 병합
        temporary_df = pd.concat([temporary_df, track_df])
        
        # 중복이 제거된 DataFrame 저장
        temporary_df.drop_duplicates(subset='track_id', inplace=True)  # track_id를 기준으로 중복 제거
        
        # 트랙 CSV 파일 저장
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        temporary_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_track_{timestamp}.csv', index=False)
        
 

2023_k-pop: 다운로드
2023_acoustic: 다운로드
2023_club: 다운로드
2023_dance: 다운로드
2023_edm: 다운로드
2023_disco: 다운로드
2023_electronic: 다운로드
2023_happy: 다운로드
2023_hip-hop: 다운로드
2023_party: 다운로드
2023_pop: 다운로드
2023_r-n-b: 다운로드
2023_rock: 다운로드
2023_romance: 다운로드
2023_sleep: 다운로드
2023_soul: 다운로드
2022_k-pop: 다운로드
2022_acoustic: 다운로드
2022_club: 다운로드
2022_dance: 다운로드
2022_edm: 다운로드
2022_disco: 다운로드
2022_electronic: 다운로드
2022_happy: 다운로드
2022_hip-hop: 다운로드
2022_party: 다운로드
2022_pop: 다운로드
2022_r-n-b: 다운로드


KeyboardInterrupt: 

In [ ]:
# 아티스트 파일을 위한 1차 컬럼 필터링
track_df2 = temporary_df[['artist_id', 'artist_name']]

print("오디오 피쳐 2분 대기")
time.sleep(10)

artist_popularity = []
artist_genres = []
artist_followers = []

for a_id in track_df.artist_id:
    artist = sp.artist(a_id)
    artist_popularity.append(artist['popularity'])
    artist_genres.append(artist['genres'])
    artist_followers.append(artist['followers']['total'])

# 아티스트 파일을 위한 최종 컬럼 (이걸로 CSV 파일 만들기)
track_df3 = track_df2.assign(artist_popularity=artist_popularity, artist_followers=artist_followers, artist_genres=artist_genres)

# 아티스트 CSV 파일 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
track_df3.to_csv(f'/home/ubuntu/project/4.5HZ/Data/test/{year}_{genre}_artist_{timestamp}.csv', index=False)